<a href="https://colab.research.google.com/github/YBatyushkova/FOR_labs/blob/main/Lab_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMPL exercises

## Previous years exams

**1) 01.2019**

Write the definition in the AMPL modelling language of the following:

* a set $I$ of all the integers between 1 and 10.
* a positive variable $x_i$ with $i ∈ {I ∪ 0}$.
* a parameter $b_{i,j}$ with $i ∈ I$ and $j ∈ {I ∪ 0}$.

Then using the defined variables and parameters write in the AMPL modelling language the following **constraint**:

$$
\sum_{i \in I}\sum_{j \in J} b_{i,j}x_j \le \sum_{i \in I}b_{i,0}x_0
$$

**2) 02.2019**

Write the definition in the AMPL modelling language of the following:
* a set $I$.
* a set $J$.
* a parameter $b_{i,j}$ with $i ∈ I$ and $j ∈ J$.
* an integer variable $x_{i,j}$ with $i ∈ I$ and $j ∈ J$.

Then write an **objective function** that maximizes the following quantity:


$$
\sum_{i \in I, j \in J, |b_{i,j} > 0|}x_{i,j}
$$

**3) 07.2019**

Write the definition in the AMPL modelling language of the following:
* a set $I$ of all the integers between 1 and 10.
* a set $J$ of all the integers between 7 and 30.
* a positive variable xij with $i ∈ I$ and $j ∈ J$.
* a parameter $b_j$ with $j ∈ J$.

Then using the defined variables and parameters write in the AMPL modelling language the following **constraint**:

$$
\sum_{i \in I:i \le b_j}x_{ij} \le 5 \qquad \forall j \in J
$$

**4) 09.2019**

Write the definition in the AMPL modelling language of the following:
* a set $I$.
* a set $J$ of all the integers between 1 and 10.
* a positive variable $x_{ij}$ with $i ∈ I$ and $j ∈ I$.
* a parameter $b_j$ with $j ∈ J$.

Then using the defined variables and parameters write in the AMPL modelling language an **objective function** that maximizes the following quantity:

$$
max \sum_{i \in I}\sum_{j \in J:i \not = j} b_jx_{ij}
$$

**5) 07.2021**

Write the definition in the AMPL modelling language of the following:
* two parameters $a$ and $b$.
* a set $I$ of all the integers between $a$ and $b$ (assume that $a < b$).
* a binary variable $x_{i,j}$ with $i, j ∈ I$.
* a parameter $b_i$ with $i ∈ I$.

Then using the defined variables and parameters write in the AMPL modelling language the following **constraint**:

$$
\sum_{j \in I:i \not = j} x_{i,j} \le 5 \qquad \forall i \in I, b_i > 1
$$

## Example WeBeep (2021)

The director of a theater must organize the playbill for the upcoming season. A total of $m$ plays must be organized (described by the set $P = 1, . . . , m$). The theater offers several types of performances, described by the set $S$.

The shows are organized by $n$ companies (described by the set $C = 1, . . . , n$). Each company $i ∈ C$ specializes in a particular type of show $a_i$ and is willing to perform up to $l_i$ shows in the upcoming season.

To guarantee the variety of the season, the theater must have at least $k_j$ planned shows of each type in the playbill $j ∈ S$.

Some time-slots of the playbill ($L ⊂ P$ is the set of the plays that have to follow this criteria) are scheduled to occur very late at night. To respect the sleep of local residents, the director decided to not schedule any concert ($a_i$ = “concert”) in these slots.

For each company $i ∈ C$ we know the staging cost $c_i$ and the expected
revenue $r_i$ of the show. Write the linear programming model of the problem of
deciding which shows to put on the bill with the objective of maximizing the
balance between profits and costs.

## Website (2007)

**1)** A small cheese factory produces m kinds of cheese. To produce one kg of cheese j, L(jI) litres of full cream milk and LjS litres of skimmed milk are needed, where j=1,…,m. The factory can produce at most Q kg per day. For the next n days, the demand of cheese j, with j=1,…,m, for each day t=1,…,n, is djt.
The maximum amount of available full cream and skimmed milk per day t=1,…,n is bIt and bSt, respectively. Note that the milk available in day t which is not used in day t cannot be used in the successive days. The factory has a warehouse (empty at the beginning of the considered time horizon) whose capacity is K.
In the warehouse, the produced cheese can be stored to be sold afterwards. The factory must meet the demand exactly. For each cheese j, with j=1,…,m, a daily storing cost per kg is given, denoted by cj. Formulate the problem of minimizing the storing cost, while satisfying the constraint on the available resources, on the production capacity and on the warehouse, and meeting the demand every day.
Formulate the problem as a Linear Programming problem.

**2)** The director of a sailing summer school received A applications for enrollment for the basic course and and applications for enrollment for the advanced course. The school has S skippers who can take both the basic and advanced courses, but who can only be assigned to one course at a time. The basic course takes place in a group: each skipper j..,S, can handle at most b persone people in its basic course.
Advanced courses, on the other hand, require each student to have a skipper who is dedicated only to him.
The school director must decide which applications to accept, which skippers to hire and how to assign pupils to skippers. The school must ensure that at least K questions for the advanced course are met. Knowing that enrollment in a basic course costs r, that in an advanced course costs c and that the salary of a skipper destined for a basic course is g, while it is l if the skipper is destined for an advanced course, write the model in linear programming of the problem of maximizing the difference between earnings and costs.

## Lab project (2021)

A company is planning on installing a series of unstaffed mini-markets in a rural area. This way, citizens that live far
from conventional stores may buy their groceries without needing the car. Specifically, the company wants to serve a set
of n houses whose coordinates in kilometers are described by parameters Cx and Cy. Some, but not all of the houses, gave
permission to the company to build a mini-marked on their property in exchange for a small compensation. Whether
or not it is possible to build a mini-market at house i is described by parameter usable[i]. All considered, the cost
of building a mini-market at house i is Dc[i]. The company wants to plan the construction of mini-markets so that
each house is at most range kilometers away (euclidean distance) from a mini-market and, the total construction cost is
minimized.

# Python exercises

## Lab project (2022)

### Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

## Exercises another group

### Diet problem

A canteen has to plan the composition of the meals that it provides. A meal can be composed of the types of food indicated in the following table. Costs, in Euro per hg, and availabilities, in hg, are also indicated.

Food|Cost|Availability
----|----|------------
Bread|0.1|4
Milk|0.5|3
Eggs|0.12|1
Meat|0.9|2
Cake|1.3|2

A meal must contain at least the following amount of each nutrient

Nutrient|Minimal quantity
--------|----------------
Calories|600 cal
Proteins|50 g
Calcium|0.7 g

Each hg of each type of food contains to following amount of nutrients

Food|Calories|Proteins|Calcium
----|--------|--------|-------
Bread|30 cal|5 g|0.02 g
Milk|50 cal|15 g|0.15 g
Eggs|150 cal|30 g|0.05 g
Meat|180 cal|90 g|0.08 g
Cake|400 cal|70 g|0.01 g

Give a linear programming formulation for the problem of finding a diet (a meal) of minimum total cost which satisfies the minimum nutrient requirements.

### Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10